In [1]:
import pm4py
import sys
import random

import pandas as pd
sys.path.append("../../TaskExecutionTimeMining")

from event_log_transformer import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [2]:
file_path = "../../../data/pcr_event_log.csv"
event_log = pd.read_csv(file_path)
event_log['time:timestamp'] = pd.to_datetime(event_log['time:timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

In [3]:
start_end_event_log2 = TransformEventLog.start_end_event_log_mult(event_log,
                                                                 start_name_1 = 'start',
                                                                 complete_name_1 = 'complete',
                                                                 merge_activity_on = ['concept:instance', 'concept:name'],
                                                                 unique_column='cpee:activity_uuid')


start_end_event_log2 = TransformEventLog.seconds_in_day(start_end_event_log2, 'time:timestamp_start')
start_end_event_log2 = TransformEventLog.day_of_week(start_end_event_log2, 'time:timestamp_start')


/Users/ferdinandfuchs/Documents/TaskExecutionTimeMining-gamlss/src/notebooks/PCR_indexbased/../../TaskExecutionTimeMining/event_log_transformer.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log[timestamp_name + complete_name_gen] - start_end_event_log[timestamp_name + start_name_gen]
/Users/ferdinandfuchs/Documents/TaskExecutionTimeMining-gamlss/src/notebooks/PCR_indexbased/../../TaskExecutionTimeMining/event_log_transformer.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [4]:
start_end_event_log2 = start_end_event_log2.rename(columns={'concept:instance': 'case:concept:name'})
transformer = TransformEventLog()
start_end_event_log2 = transformer.index_based_encode(
    start_end_event_log2,
    ['concept:name'],
    max_prefix_len=3
)

In [5]:
random.seed(42)
train, test = pm4py.ml.split_train_test(start_end_event_log2, case_id_key='case:concept:name')

In [6]:
print(train.shape, test.shape)
print(train['case:concept:name'].unique().shape, test['case:concept:name'].unique().shape)

(14720, 7) (3570, 7)
(4960,) (1202,)


In [7]:
train.to_csv('../../../data/PCR/transformed_event_logs_indexbased/PCR_train.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')
train.to_pickle('../../../data/PCR/transformed_event_logs_indexbased/PCR_train.pickle')

In [8]:
test.to_csv('../../../data/PCR/transformed_event_logs_indexbased/PCR_test.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')
train.to_pickle('../../../data/PCR/transformed_event_logs_indexbased/PCR_test.pickle')